In [8]:
# 先產生一個CSV檔案，給後面的程式讀取
import json
import pandas as pd
import os
import re

# 設定資料夾路徑
directory_path = "./docs/output/1_clean_json/llama"
# 儲存為 CSV 檔案
csv_path = "./docs/output/2_metadata/processed_data.csv"

# 讀取 JSON 檔案
def read_json(file_path):
    try:
        with open(file_path, 'r', encoding='utf-8') as file:
            data = json.load(file)
        return data
    except json.JSONDecodeError as e:
        print(f"Error decoding JSON in file {file_path}: {e}")
        return None
    except Exception as e:
        print(f"Error reading file {file_path}: {e}")
        return None

# 目標關鍵字正則表達式（優先處理 "檔案來源機關"）
target_patterns = {
    "案名": r"(案名：)",
    "檔案內容": r"(檔案內容：)",
    "檔號": r"(檔號：)",
    "管有機關": r"(管有機關：)",
    "檔案來源機關": r"(檔案來源機關：)",  # **優先處理**
    "來源機關": r"(來源機關：)"  # **如果沒有 "檔案來源機關" 才處理**
}

# 處理 `reference` 欄位
def process_reference(reference):
    if isinstance(reference, list):
        modified_ref_list = []
        contains_file_source = any("檔案來源機關" in ref for ref in reference)  # 是否有 "檔案來源機關"

        for ref in reference:
            if any(key in ref for key in target_patterns):  # 只處理包含關鍵字的行
                for key, pattern in target_patterns.items():
                    # **如果"檔案來源機關"已經存在，就不再處理 "來源機關"**
                    if key == "來源機關" and contains_file_source:
                        continue
                    ref = re.sub(pattern, r" | \1", ref)  # 在關鍵字前加 " | "
            modified_ref_list.append(ref.strip())  # 去除前後空白

        return modified_ref_list
    return reference  # 非列表的情況，直接返回

# 處理 JSON 數據
def process_data(data, file_id):
    file_ids = []  # 每個 reference 對應的 file_id
    references = []  # 處理後的 reference

    if isinstance(data, dict):
        data = [data]
    
    if isinstance(data, list):
        for item in data:
            title = item.get('titles', '')
            if isinstance(title, list):
                title = title[0] if title else ''
            
            reference = item.get('reference', [])
            modified_reference = process_reference(reference)

            for ref in modified_reference:
                file_ids.append(file_id)
                references.append(ref)  # 每個 reference 獨立存入

    # 創建 DataFrame
    df = pd.DataFrame({
        'file_id': file_ids,
        'reference': references
    })
    
    return df

# 讀取所有 JSON 檔案
def read_all_json_files(directory_path):
    all_data = []
    
    for filename in os.listdir(directory_path):
        if filename.endswith('.json'):
            file_path = os.path.join(directory_path, filename)
            print(f"Reading file: {file_path}")
            data = read_json(file_path)
            if data:
                all_data.append((data, filename))
    
    return all_data

# 讀取所有 JSON 檔案
all_data = read_all_json_files(directory_path)

# 整理資料
all_file_ids = []
all_references = []

# 處理所有檔案的資料
for data, file_id in all_data:
    df = process_data(data, file_id)
    all_file_ids.extend(df['file_id'])
    all_references.extend(df['reference'])

# 創建最終 DataFrame
final_df = pd.DataFrame({
    'file_id': all_file_ids,
    'reference': all_references
})

final_df.to_csv(csv_path, index=False, encoding='utf-8-sig')

print(f"CSV file has been saved to {csv_path}")




Reading file: ./docs/output/1_clean_json/llama\228事件(20).json
Reading file: ./docs/output/1_clean_json/llama\「友仔」是什麼？光復初期臺北地區非法組織調查報告告訴您(37).json
Reading file: ./docs/output/1_clean_json/llama\「回首向來蕭瑟處，歸去，也無風雨也無晴」—民國38年國軍遷臺紀事(30).json
Reading file: ./docs/output/1_clean_json/llama\「威海衛」租借地的收回(42).json
Reading file: ./docs/output/1_clean_json/llama\「快速」發展的年代：麥克阿瑟公路通車一甲子(203).json
Reading file: ./docs/output/1_clean_json/llama\「日暮鄉關何處是」─「留越國軍」的返台路(31).json
Reading file: ./docs/output/1_clean_json/llama\「賽德克．巴萊」重現的霧社事件(17).json
Reading file: ./docs/output/1_clean_json/llama\「醫者仁也‧仁者人也」─光復初期臺灣醫學教育(29).json
Reading file: ./docs/output/1_clean_json/llama\ㄋㄟㄋㄟ補給站：美援牛奶的供應(127).json
Reading file: ./docs/output/1_clean_json/llama\一紙命令，臺灣命運大不同─中國台灣省行政長官公署警備總司令部第一號令(1).json
Reading file: ./docs/output/1_clean_json/llama\不用手機也可哈拉一整天─45年度公用電話擴充計畫(2).json
Reading file: ./docs/output/1_clean_json/llama\不能少了你—臺灣光復後首次戶口清查(35).json
Reading file: ./docs/output/1_clean_json/llama\世界人權日(18).json
Reading fil

In [9]:
import csv
import json
import os
import re

# 設定路徑

#directory_path = "./docs/output/1_clean_json/llama"
#csv_file_path  = csv_path
output_dir = "./docs/output/2_metadata/v3"

# 確認輸出目錄是否存在，若不存在則創建
os.makedirs(output_dir, exist_ok=True)

def detect_delimiter(csv_path):
    with open(csv_path, "r", encoding="utf-8-sig") as file:
        sample = file.readline()
        if "\t" in sample:
            return "\t"
        elif ";" in sample:
            return ";"
        else:
            return ","

delimiter = detect_delimiter(csv_path)

def parse_csv_to_json(csv_path, delimiter):
    data_dict = {}

    with open(csv_path, "r", encoding="utf-8-sig") as file:
        reader = csv.reader(file, delimiter=delimiter)
        for row in reader:
            if len(row) < 2:
                continue

            file_name = row[0].strip()
            details = row[1].strip()

            # 檢查 "檔號" 是否存在於該行，若無則跳過
            if "檔號" not in details:
                continue

            #print(f"處理檔案：{file_name}，詳情：{details}")

            # 初始化 JSON 結構
            if file_name not in data_dict:
                data_dict[file_name] = {"metadata": []}

            # 解析 "檔案內容"（從「」中提取內容）
            content_match = re.search(r"檔案內容：([^ ]+)", details)

            # 解析 "檔案管有機關" 和 "檔案來源機關"
            source_match = re.search(r"檔案管有機關：([^ ]+) 檔案來源機關：([^ ]+)", details)

            # 解析其他欄位（檔號、來源機關、管有機關、案名）
            entry = {"案名": None}  # 預設 "案名": null
            for key in ["檔號", "來源機關", "管有機關", "案名"]:
                match = re.search(fr"{key}：([^|]+)", details)
                if match:
                    entry[key] = match.group(1).strip()

            # 如果有找到檔案管有機關和檔案來源機關
            if source_match:
                entry["檔案管有機關"] = source_match.group(1).strip()
                entry["檔案來源機關"] = source_match.group(2).strip()

            # 加入 "檔案內容"
            if content_match:
                entry["檔案內容"] = content_match.group(1).strip()

            # 檢查 entry 是否為空，若非空則加入
            if any(value is not None for value in entry.values()):  # 確保至少有一個有效值
                data_dict[file_name]["metadata"].append(entry)

    return data_dict

count = 0  # 初始化處理檔案數量計數器

# 讀取 CSV 資料並解析
parsed_data = parse_csv_to_json(csv_path, delimiter)

# 讀取 JSON 檔案並與 CSV 資料合併
for json_file_name in os.listdir(directory_path):
    if json_file_name.endswith(".json"):
        json_file_path = os.path.join(directory_path, json_file_name)

        # 嘗試讀取 JSON 檔案
        try:
            with open(json_file_path, "r", encoding="utf-8") as json_file:
                json_data = json.load(json_file)

            # 確認是否有對應的資料
            if json_file_name in parsed_data:
                # 合併資料（僅保留 reference 和 metadata 欄位）
                json_data = {
                    "reference": json_data.get("reference", []),
                    "metadata": parsed_data[json_file_name].get("metadata", [])
                }

                # 輸出到新的 JSON 檔案
                output_json_path = os.path.join(output_dir, json_file_name)
                with open(output_json_path, "w", encoding="utf-8") as json_output_file:
                    json.dump(json_data, json_output_file, ensure_ascii=False, indent=4)
                print(f"✅ 檔案已儲存到 {output_json_path}")
            else:
                # 沒有找到對應的 CSV 資料，將原始 JSON 寫入新目錄
                print(f"⚠️ 原始JSON檔案 {json_file_name} 不存在檔號資訊。")
                output_json_path = os.path.join(output_dir, json_file_name)
                with open(output_json_path, "w", encoding="utf-8") as json_output_file:
                    json.dump(json_data, json_output_file, ensure_ascii=False, indent=4)

                print(f"✅ 原始 JSON 檔案已儲存到 {output_json_path}")
            count += 1  # 成功處理一個檔案後，計數 +1

        except Exception as e:
            print(f"❌ 讀取檔案 {json_file_name} 時發生錯誤: {e}")
            
print(f"\n📂 共處理完成 {count} 個檔案")

✅ 檔案已儲存到 ./docs/output/2_metadata/v3\228事件(20).json
✅ 檔案已儲存到 ./docs/output/2_metadata/v3\「友仔」是什麼？光復初期臺北地區非法組織調查報告告訴您(37).json
✅ 檔案已儲存到 ./docs/output/2_metadata/v3\「回首向來蕭瑟處，歸去，也無風雨也無晴」—民國38年國軍遷臺紀事(30).json
✅ 檔案已儲存到 ./docs/output/2_metadata/v3\「威海衛」租借地的收回(42).json
✅ 檔案已儲存到 ./docs/output/2_metadata/v3\「快速」發展的年代：麥克阿瑟公路通車一甲子(203).json
✅ 檔案已儲存到 ./docs/output/2_metadata/v3\「日暮鄉關何處是」─「留越國軍」的返台路(31).json
✅ 檔案已儲存到 ./docs/output/2_metadata/v3\「賽德克．巴萊」重現的霧社事件(17).json
✅ 檔案已儲存到 ./docs/output/2_metadata/v3\「醫者仁也‧仁者人也」─光復初期臺灣醫學教育(29).json
✅ 檔案已儲存到 ./docs/output/2_metadata/v3\ㄋㄟㄋㄟ補給站：美援牛奶的供應(127).json
⚠️ 原始JSON檔案 一紙命令，臺灣命運大不同─中國台灣省行政長官公署警備總司令部第一號令(1).json 不存在檔號資訊。
✅ 原始 JSON 檔案已儲存到 ./docs/output/2_metadata/v3\一紙命令，臺灣命運大不同─中國台灣省行政長官公署警備總司令部第一號令(1).json
✅ 檔案已儲存到 ./docs/output/2_metadata/v3\不用手機也可哈拉一整天─45年度公用電話擴充計畫(2).json
✅ 檔案已儲存到 ./docs/output/2_metadata/v3\不能少了你—臺灣光復後首次戶口清查(35).json
✅ 檔案已儲存到 ./docs/output/2_metadata/v3\世界人權日(18).json
✅ 檔案已儲存到 ./docs/output/2_metadata/v3\世紀糖鐵穿鄉越鎮的五分車(148).json
✅ 檔案已儲存到 

In [10]:
import os
import json

json_dir = output_dir  # JSON檔案所在資料夾

# 取得所有 JSON 檔案
files = [f for f in os.listdir(json_dir) if f.endswith(".json")]

for file in files:
    full_path = os.path.join(json_dir, file)
    
    with open(full_path, "r", encoding="utf-8") as f:
        data = json.load(f)
        
        print(f"\n===== 檔案名稱: {file} =====")
        print(json.dumps(data, indent=2, ensure_ascii=False))


===== 檔案名稱: 228事件(20).json =====
{
  "reference": [
    "檔案管有機關：檔案管理局 檔案來源機關：臺灣省菸酒公賣局 檔案內容：查緝天馬茶房一事私煙處理情形 檔號：0036/523.3/9/1/002",
    "參考資料：檔案管理局虛擬檔案館。〈228事件主題區〉，http://va.archives.gov.tw/main.asp(民98年1月22日檢索)。",
    "檔案管有機關：檔案管理局 檔案來源機關：臺灣省菸酒公賣局 檔案內容：查緝天馬茶房一事私煙處理情形 檔號：0036/523.3/9/1/002",
    "參考資料：檔案管理局虛擬檔案館。〈228事件主題區〉，http://va.archives.gov.tw/main.asp(民98年1月22日檢索)。",
    "政府網站資料開放宣告|隱私權、著作權及資訊安全政策宣告 | 本局電子信箱"
  ],
  "metadata": [
    {
      "案名": null,
      "檔號": "0036/523.3/9/1/002",
      "來源機關": "臺灣省菸酒公賣局",
      "管有機關": "檔案管理局",
      "檔案內容": "查緝天馬茶房一事私煙處理情形"
    },
    {
      "案名": null,
      "檔號": "0036/523.3/9/1/002",
      "來源機關": "臺灣省菸酒公賣局",
      "管有機關": "檔案管理局",
      "檔案內容": "查緝天馬茶房一事私煙處理情形"
    }
  ]
}

===== 檔案名稱: 「友仔」是什麼？光復初期臺北地區非法組織調查報告告訴您(37).json =====
{
  "reference": [
    "參考資料：",
    "1.臺灣省行政長官公署編。《臺灣省行政長官公署三月來工作概要（三十四年十月二十五日—三十五年一月二十四日）》。臺北市：編者，民35。",
    "2.黃玉齋主編，《臺灣年鑑》。臺北市：海峽學術出版社，民90。",
    "3.黃靜嘉。《春帆樓下晚濤急：日本對臺灣殖民統治及其影響》。臺北市：臺灣商務，民91。",
    "檔案影像1-1 檔